In [1]:
!pip -q install chromadb openai langchain tiktoken

In [4]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import requests
import zipfile
import io

# URL of the file to be downloaded
url = "https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip?dl=1"

# Download the file
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Unzip the file
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        zip_ref.extractall("new_articles")
    print("Download and extraction complete.")
else:
    print(f"Failed to download file. HTTP Status Code: {response.status_code}")


Download and extraction complete.


In [8]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [13]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
!pip install unstructured


  Obtaining dependency information for unstructured from https://files.pythonhosted.org/packages/d8/28/8debc94107c06a42cf1d3e2e62ab8642b38d2282ea3595e3038259e61acd/unstructured-0.14.4-py3-none-any.whl.metadata
  Obtaining dependency information for python-magic from https://files.pythonhosted.org/packages/6c/73/9f872cb81fc5c3bb48f7227872c28975f998f3e7c2b1c16e95e6432bbb90/python_magic-0.4.27-py2.py3-none-any.whl.metadata
  Obtaining dependency information for emoji from https://files.pythonhosted.org/packages/e6/90/20ad30babfa8f2b5ab46281d8e17bdfdbb3ac294cda14d525b9c2d958846/emoji-2.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for python-iso639 from https://files.pythonhosted.org/packages/01/08/5e649cf18dec750d498c53c6c8eb1d9790752ebd50fa7f7e69cc0c277cfe/python_iso639-2024.4.27-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ------------------------------ ------ 798.7/981.5 kB 16.8 MB/s eta 0:00:01
     -------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiohttp 3.8.3 requires charset-normalizer<3.0,>=2.0, but you have charset-normalizer 3.3.2 which is incompatible.
botocore 1.27.59 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.1 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.
spacy 3.6.1 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.


In [14]:
loader = DirectoryLoader("C:/Users/sruja/2nd year/Udemy course/new_articles",glob = "./*.txt")

In [16]:
document = loader.load()

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 100)
text = text_splitter.split_documents(document)

In [21]:
text[0].page_content

'Signaling that investments in the supply chain sector remain robust, Pando, a startup developing fulfillment management technologies, today announced that it raised $30 million in a Series B round, bringing its total raised to $45 million.\n\nIron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.\n\n“We will not expand into new industries or adjacent product areas,” he told TechCrunch in an email interview. “Great talent is the foundation of the business — we will continue to augment our teams at all levels of the organization. Pando is also open to exploring strategic partnerships and acquisitions with this round of funding.”'

In [22]:
from langchain import embeddings

In [23]:
persist_directory = 'db'

In [28]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [32]:
vectordb = Chroma.from_documents(documents=text, embedding=embeddings, persist_directory=persist_directory)

In [33]:
vectordb.persist()

C:\Users\sruja\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [34]:
vectordb = None

In [35]:
vectordb = Chroma(persist_directory=persist_directory,embedding_function=embeddings)

In [43]:
retriever = vectordb.as_retriever()

In [44]:
docs = retriver.get_relevant_documents("how much money did microsoft raise?")

In [45]:
docs

[Document(page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enterprises seeking to manage their end users.”', metadata={'source': 'C:\\Users\\sruja\\2nd year\\Udemy course\\new_articles\\05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}),
 Document(page_conte

In [39]:
from langchain.chains import RetrievalQA

In [40]:
llm=OpenAI()

C:\Users\sruja\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [46]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [47]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response['source_documents']:
        print(source.metadata['source'])

In [48]:
query = "how much money did microsoft raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

C:\Users\sruja\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 $10 billion


Sources:
C:\Users\sruja\2nd year\Udemy course\new_articles\05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt
C:\Users\sruja\2nd year\Udemy course\new_articles\05-07-3one4-capital-driven-by-contrarian-bets-raises-200-million-new-fund.txt
C:\Users\sruja\2nd year\Udemy course\new_articles\05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt
C:\Users\sruja\2nd year\Udemy course\new_articles\05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt
